In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('movie_rv.csv')

In [2]:
df

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...,...
149995,149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,149996,8549745,평점이 너무 낮아서...,1
149997,149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


### 텍스트 분석을 통한 -> 코사인 유사도 살펴보기!

In [5]:
df_sp=df[:50000]

In [8]:
df_sp=df_sp.dropna()

In [9]:
#na값 정리 1개 있었음
df_sp.isna().sum()

Unnamed: 0    0
id            0
document      0
label         0
dtype: int64

- 리뷰 데이터를 학습시키기 위해서는 어떤 변환을 할까?
- 수치형 데이터 처럼 변환해야 한다. ( 흔히 말하는 인코딩 작업, 임베딩 작업 )

- 사이킷런에서 제공하는 counterVectorizer 쉽게 만들어주는 형태 

In [19]:
df_sp_rv= df_sp['document']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# 유사도를 계산하기 위한 학습을 위해 작업 진행
# Count 카운팅으로 진행하는 것 

In [12]:
rv_cv=CountVectorizer(max_features=1000)

In [13]:
rv_cv # 1,000개의 기준은 가장 카운팅 순서로 봤을 때 많은 것부터해서 1,000개까지만 사용하겠다.


CountVectorizer(max_features=1000)

In [20]:
df_sp_rv

0                                      아 더빙.. 진짜 짜증나네요 목소리
1                        흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                        너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4        사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                               ...                        
49995    "이게 소위 스페인식 Neo-realism 이란 건가? - 물론, ""아동 판타지"...
49996                                     보지마셈 저 믿으세요 보지마셈
49997                                                 최고졸작
49998                                    재밌고 좋았음 10자 쓰라고?!
49999                                 소박하지만 잔잔한 감동을 주는 영화.
Name: document, Length: 49999, dtype: object

In [21]:
re_mv=rv_cv.fit_transform(df_sp_rv) #리뷰학습

In [22]:
re_mv

<49999x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 124124 stored elements in Compressed Sparse Row format>

In [25]:
print(rv_cv.get_feature_names_out()[:50])

['0점' '0점은' '10' '10자' '10점' '1점' '1점도' '2점' '3류' '3점' '4점' '80년대' '90년대'
 '9점' 'b급' 'good' 'oo' 'ooo' 'ooo기' 'ost' 'the' 'tv' 'ㄷㄷ' 'ㅅㅂ' 'ㅇㅇ' 'ㅉㅉ'
 'ㅋㅋ' 'ㅋㅋㅋ' 'ㅋㅋㅋㅋ' 'ㅋㅋㅋㅋㅋ' 'ㅎㅎ' 'ㅎㅎㅎ' 'ㅜㅜ' 'ㅠㅠ' 'ㅠㅠㅠ' 'ㅡㅡ' '가끔' '가는' '가볍게'
 '가서' '가슴' '가슴이' '가장' '가지고' '가진' '각본' '간만에' '갈수록' '감독' '감독님']


In [26]:
rv_cv

CountVectorizer(max_features=1000)

In [27]:
(49999/(124124*1000)*100)

0.04028149270084754

- 텍스트 분석전에 전처리 작업을 했다.
- 전처리 작업은 예전에 배웠던 Okt를 이용해서 진행

In [28]:
from konlpy.tag import Okt

In [29]:
tw_tag= Okt()

In [31]:
# 전체 데이터 셋을 전처리 하기 위한 작업

def rv_tokenizer(doc):
    return [
        token for token, pos in tw_tag.pos(doc)
        if pos in ['Noun','Verb']
    ]


In [32]:
rv_tokenizer(df_sp_rv[1])

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '않구나']

In [33]:
# 전처리를 원하는 함수를 만들어서 CountVectorizer 사용이 가능하다

re_cv_okt = CountVectorizer(max_features=1000, tokenizer =rv_tokenizer )

In [35]:
rev_cv=re_cv_okt.fit_transform(df_sp_rv)

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
print(re_cv_okt.get_feature_names_out()[:50])

['가' '가끔' '가는' '가면' '가서' '가슴' '가요' '가장' '가족' '가지' '가진' '가치' '각본' '간' '간다'
 '간만' '갈수록' '감' '감독' '감동' '감명' '감사합니다' '감상' '감성' '감정' '갑자기' '강' '강추' '개'
 '개그' '개봉' '개연' '개인' '개판' '거' '거기' '거리' '거의' '거지' '건' '건가' '건지' '걸' '걸까'
 '걸작' '겁나' '것' '게' '게이' '게임']


In [37]:
df_sp_rv

0                                      아 더빙.. 진짜 짜증나네요 목소리
1                        흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                        너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4        사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                               ...                        
49995    "이게 소위 스페인식 Neo-realism 이란 건가? - 물론, ""아동 판타지"...
49996                                     보지마셈 저 믿으세요 보지마셈
49997                                                 최고졸작
49998                                    재밌고 좋았음 10자 쓰라고?!
49999                                 소박하지만 잔잔한 감동을 주는 영화.
Name: document, Length: 49999, dtype: object

- 49,999 행 , 1,000개 컬럼 열
    - 가, 가끔, 가는, 가면
- 0  0 ,  1 , 0 , 0
- 1  0,  0 , 0 , 0 ,,
- 2  1, 1, 
- 0,1 카운팅이 진행



In [40]:
rev_cv

<49999x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 227438 stored elements in Compressed Sparse Row format>

### 코사인 유사도 계산

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
df_sp_rv

0                                      아 더빙.. 진짜 짜증나네요 목소리
1                        흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                        너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4        사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                               ...                        
49995    "이게 소위 스페인식 Neo-realism 이란 건가? - 물론, ""아동 판타지"...
49996                                     보지마셈 저 믿으세요 보지마셈
49997                                                 최고졸작
49998                                    재밌고 좋았음 10자 쓰라고?!
49999                                 소박하지만 잔잔한 감동을 주는 영화.
Name: document, Length: 49999, dtype: object

In [44]:
df_sp_rv[3]

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [47]:
# 코사인 유사도를 학습하며 비교할 기준이 필요해서
st = len(df_sp_rv[3])//2

# 중심점으로부터 뒤 절반으로 가지고와서 비교할 문서 생성

source = df_sp_rv[3][-st:]

In [48]:
source

' 재미는 없다..평점 조정'

In [51]:
#유사도 계산하기 전에 벡터로 변환하자

source_rv=re_cv_okt.transform([source])

In [53]:
source_rv

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [54]:
rev_cv

<49999x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 227438 stored elements in Compressed Sparse Row format>

In [56]:
sim_value=cosine_similarity(source_rv,rev_cv)

In [61]:
sim_value[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [62]:
# 유사도 계산을 정렬하여서 확인해 보자!
print('유사도 정렬',sorted(sim_value[0], reverse=True)[:1])

유사도 정렬 [0.9999999999999998]


In [63]:
print('가장 유사한 리뷰 인덱스?',np.argmax(sim_value[0]))

가장 유사한 리뷰 인덱스? 49685


In [65]:
df_sp_rv[3]

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [64]:
#49685 코사인 값이 가장 높다
df_sp_rv[49685]

'2012년에 본 90년대 연출. 잘봤네요. 향수에 젖었어요. 시바. 웜홀을 타고 넘어와서 만들었나....연출공부좀 더 하쇼.'

### 카운트 벡터를 사용하여 학습 시킨 것
- TF-IDF (Term Frequency - Inverse Document Frequency) 
- 단어가 더 많은 문서에서 나타날수록 그 단어는 별로 중요하지 않을 수 있다.
- 빈도로 계산하여서 새롭게 계산하는 방법
- 단어빈도- 역문서빈도 

In [74]:
from sklearn.feature_extraction.text import TfidfTransformer

In [75]:
# 동일하게 변환작업
tfidf=TfidfTransformer()

In [80]:
tfidf_rev_cv=tfidf.fit_transform(rev_cv) # 바로 리뷰데이터를 넣는 게 아니라, 전처리하여 만든 메트릭스를 넣어서 학습시킨다. 

In [85]:
rev_cv[10].toarray()[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [86]:
tfidf_rev_cv[10].toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [89]:
source_rv_tfidf=tfidf.fit_transform(source_rv) # 소스데이터도 (기준데이터) Tfidffh 변환

In [90]:
sim_value_tfidf=cosine_similarity(source_rv_tfidf,tfidf_rev_cv)

In [91]:
sim_value_tfidf

array([[0., 0., 0., ..., 0., 0., 0.]])

In [92]:
print('가장 유사한 리뷰 인덱스?',np.argmax(sim_value_tfidf[0]))

가장 유사한 리뷰 인덱스? 49685


In [93]:
#49685
df_sp_rv[49685]

'2012년에 본 90년대 연출. 잘봤네요. 향수에 젖었어요. 시바. 웜홀을 타고 넘어와서 만들었나....연출공부좀 더 하쇼.'

In [96]:
(-sim_value_tfidf[0]).argsort()[:10]

array([49685, 40904, 10773, 26831,     3,   221, 42521, 12735, 21135,
       21139])

In [97]:
df_sp_rv[40904]

'지루하고.. 그닥...별로 감정이 공감가지않는 영화'

In [98]:
df_sp_rv[10773]

'평점좋아 봤는데 완젼 재미 더럽게 없다.'

In [99]:
df_sp_rv[26831]

'너무 재밌고 깨닫는게 많은 영화임'

In [100]:
df_sp_rv[221]

'진심 재미 없는데 너무 평점 높아서 화남;'

In [101]:
(-sim_value[0]).argsort()[:10]

array([49685, 40904, 21648, 39964,   221, 10773,     3, 26831, 35304,
       31253])

In [102]:
df_sp_rv[21648]

'무섭긴한데 평점만큼 재미는 업음노잼'